In [1]:
import pandas as pd
import numpy as np

In [75]:
# Variables
USED_COLUMNS = []
UNUSED_COLUMNS = []

In [2]:
# 
Password_Attack = pd.read_csv("Password_attack.csv")
print("Password_Attack loaded!")
SQL_injection = pd.read_csv("SQL_injection_attack.csv")
print("SQL_injection loaded!")
XSS_attack = pd.read_csv("XSS_attack.csv")
print("XSS_attack loaded!")
Uploading_attack = pd.read_csv("Uploading_attack.csv")
print("Uploading_attack loaded!")

C:\Users\ParkJaehyung\AppData\Local\Temp\ipykernel_12332\1384700295.py:2: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  Password_Attack = pd.read_csv("Password_attack.csv")


Password_Attack loaded!
SQL_injection loaded!
XSS_attack loaded!
Uploading_attack loaded!


In [3]:
common_columns = []
uncommon_columns = []
for col in set([*SQL_injection.columns, *XSS_attack.columns, *Uploading_attack.columns, *Password_Attack.columns]):
    if col in SQL_injection.columns and col in XSS_attack.columns and col in Uploading_attack.columns and col in Password_Attack.columns:
        common_columns.append(col)
    else: 
        uncommon_columns.append(col)

In [4]:
uncommon_columns

[]

In [27]:
common_columns

['tcp.connection.syn',
 'dns.qry.name.len',
 'http.request.uri.query',
 'ip.src_host',
 'dns.retransmit_request',
 'mqtt.msgtype',
 'frame.time',
 'tcp.flags.ack',
 'mqtt.topic_len',
 'icmp.seq_le',
 'arp.hw.size',
 'mqtt.msg',
 'dns.qry.qu',
 'icmp.unused',
 'http.tls_port',
 'dns.retransmit_request_in',
 'tcp.srcport',
 'mqtt.len',
 'tcp.checksum',
 'Attack_label',
 'mqtt.proto_len',
 'http.response',
 'Attack_type',
 'tcp.flags',
 'dns.qry.name',
 'dns.retransmission',
 'arp.dst.proto_ipv4',
 'arp.src.proto_ipv4',
 'tcp.connection.rst',
 'mbtcp.len',
 'mqtt.ver',
 'tcp.len',
 'ip.dst_host',
 'udp.port',
 'mqtt.conack.flags',
 'http.file_data',
 'mqtt.msg_decoded_as',
 'mbtcp.unit_id',
 'tcp.ack_raw',
 'icmp.transmit_timestamp',
 'tcp.options',
 'mqtt.protoname',
 'mqtt.conflags',
 'mqtt.topic',
 'tcp.payload',
 'arp.opcode',
 'icmp.checksum',
 'dns.qry.type',
 'tcp.connection.synack',
 'http.request.version',
 'mqtt.conflag.cleansess',
 'udp.time_delta',
 'tcp.ack',
 'tcp.dstport',


In [5]:
pd.concat([SQL_injection["tcp.connection.syn"], XSS_attack["tcp.connection.syn"]]).unique().shape[0] > 1

True

In [72]:
cols = list(Password_Attack.columns)
unused = []
for col in common_columns:
    series = pd.concat([SQL_injection[col], XSS_attack[col], Uploading_attack[col], Password_Attack[col]])
    unique_datas = series.unique()
    if unique_datas.shape[0] <= 1:
        cols.remove(col)
        unused.append(col)

In [76]:
USED_COLUMNS = cols
UNUSED_COLUMNS = [*UNUSED_COLUMNS, *unused]
cols, unused


(['frame.time',
  'ip.src_host',
  'ip.dst_host',
  'arp.dst.proto_ipv4',
  'arp.opcode',
  'arp.hw.size',
  'arp.src.proto_ipv4',
  'http.file_data',
  'http.content_length',
  'http.request.uri.query',
  'http.request.method',
  'http.referer',
  'http.request.full_uri',
  'http.request.version',
  'http.response',
  'tcp.ack',
  'tcp.ack_raw',
  'tcp.checksum',
  'tcp.connection.fin',
  'tcp.connection.rst',
  'tcp.connection.syn',
  'tcp.connection.synack',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.ack',
  'tcp.len',
  'tcp.options',
  'tcp.payload',
  'tcp.seq',
  'tcp.srcport',
  'Attack_type'],
 ['dns.qry.type',
  'mqtt.conack.flags',
  'mqtt.proto_len',
  'mqtt.topic_len',
  'udp.stream',
  'mqtt.hdrflags',
  'mqtt.conflags',
  'icmp.transmit_timestamp',
  'mbtcp.len',
  'dns.retransmit_request',
  'mqtt.msgtype',
  'dns.qry.name',
  'mqtt.msg_decoded_as',
  'dns.retransmission',
  'mqtt.len',
  'mqtt.conflag.cleansess',
  'udp.port',
  'mqtt.protoname',
  'mqtt.topic',
  'A

In [9]:
import json
with open("column_data.txt", "w") as f:
    f.write(json.dumps({"Used_Columns":cols, "Unused_Columns":unused}, indent=2))

In [10]:
concat_datas = pd.DataFrame({col:pd.concat([SQL_injection[col], XSS_attack[col], Uploading_attack[col], Password_Attack[col]]) for col in cols})

In [11]:
processed_data = concat_datas

In [16]:
import json
with open("unique_values.txt", "w") as f:
    data = {}
    for col in cols:
        count = processed_data[col].unique().shape[0]
        unique_cols = list(processed_data[col].unique()) if count < 10 else list(processed_data[col].unique()[:10])
        data[col] = {
            "count": count,
            "unique_columns": unique_cols
        }
    f.write(json.dumps(data, indent=2, ensure_ascii=True))

In [63]:
temp = processed_data.loc[(processed_data["http.request.method"] != '0')]

In [77]:
temp_cols = temp.columns
used = list(temp_cols[:])
unused = []
for col in temp.columns:
    if not all(temp[col]) or temp[col].unique().shape[0] <= 1 and col != 'Attack_type':
        used.remove(col)
        unused.append(col)

In [78]:
USED_COLUMNS = used
UNUSED_COLUMNS = set([*UNUSED_COLUMNS, *unused])
used, unused

(['frame.time',
  'http.request.method',
  'http.request.full_uri',
  'http.request.version',
  'tcp.ack',
  'tcp.ack_raw',
  'tcp.checksum',
  'tcp.len',
  'tcp.options',
  'tcp.payload',
  'tcp.seq',
  'tcp.srcport',
  'Attack_type'],
 [])

In [82]:
import json
with open("column_data.txt", "w") as f:
    f.write(json.dumps({"USED_COLUMNS":USED_COLUMNS, "UNUSED_COLUMNS":list(UNUSED_COLUMNS)}, indent=2, ensure_ascii=True))    

In [66]:
temp = temp.drop(unused, axis="columns")

In [67]:
temp[:15]

,frame.time,http.request.method,http.request.full_uri,http.request.version,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.len,tcp.options,tcp.payload,tcp.seq,tcp.srcport,Attack_type
2,2021 21:35:18.571627000,GET,http://192.168.0.128/DVWA/vulnerabilities/sqli...,HTTP/1.1,1.0,1.221191e+09,0x000023bd,306.0,0101080a9b1e676b9eabb8af,474554202f445657412f76756c6e65726162696c697469...,1.0,37046.0,SQL_injection
15,2021 21:35:18.642567000,GET,http://192.168.0.128/DVWA/vulnerabilities/sqli...,HTTP/1.1,1.0,2.497556e+09,0x0000d49b,330.0,0101080a9b1e67b49eabfa17,474554202f445657412f76756c6e65726162696c697469...,1.0,37048.0,SQL_injection
24,2021 21:35:18.691481000,GET,http://192.168.0.128/DVWA/vulnerabilities/sqli...,HTTP/1.1,1.0,1.526554e+09,0x00004a68,333.0,0101080a9b1e67e59eabfa5c,474554202f445657412f76756c6e65726162696c697469...,1.0,37050.0,SQL_injection
35,2021 21:35:18.816771000,GET,http://192.168.0.128/DVWA/vulnerabilities/sqli...,HTTP/1.1,1.0,2.083381e+08,0x00002fad,331.0,0101080a9b1e68629eabfaac,474554202f445657412f76756c6e65726162696c697469...,1.0,37052.0,SQL_injection
46,2021 21:35:18.868654000,GET,http://192.168.0.128/DVWA/vulnerabilities/sqli...,HTTP/1.1,1.0,4.582203e+08,0x000045cd,305.0,0101080a9b1e68969eabfb0f,474554202f445657412f76756c6e65726162696c697469...,1.0,37054.0,SQL_injection
57,2021 21:35:18.928287000,GET,http://192.168.0.128/DVWA/vulnerabilities/sqli...,HTTP/1.1,1.0,3.881938e+09,0x0000cb97,314.0,0101080a9b1e68d29eabfb47,474554202f445657412f76756c6e65726162696c697469...,1.0,37056.0,SQL_injection
68,2021 21:35:18.980536000,GET,http://192.168.0.128/DVWA/vulnerabilities/sqli...,HTTP/1.1,1.0,3.203403e+09,0x0000725d,343.0,0101080a9b1e69069eabfb99,474554202f445657412f76756c6e65726162696c697469...,1.0,37058.0,SQL_injection
79,2021 21:35:19.029708000,GET,http://192.168.0.128/DVWA/vulnerabilities/sqli...,HTTP/1.1,1.0,1.272756e+08,0x00002a4e,337.0,0101080a9b1e69389eabfbca,474554202f445657412f76756c6e65726162696c697469...,1.0,37060.0,SQL_injection
90,2021 21:35:19.083980000,GET,http://192.168.0.128/DVWA/vulnerabilities/sqli...,HTTP/1.1,1.0,3.589361e+09,0x000076b2,406.0,0101080a9b1e696e9eabfbfd,474554202f445657412f76756c6e65726162696c697469...,1.0,37062.0,SQL_injection
100,2021 21:35:19.130055000,GET,http://192.168.0.128/DVWA/vulnerabilities/sqli...,HTTP/1.1,1.0,1.710834e+09,0x0000c7ca,439.0,0101080a9b1e699c9eabfc35,474554202f445657412f76756c6e65726162696c697469...,1.0,37064.0,SQL_injection


In [68]:
temp.to_csv("full_uri,content_length,method_na_removed.csv", sep=",")

In [69]:
temp.count()

frame.time               78697
http.request.method      78697
http.request.full_uri    78697
http.request.version     78697
tcp.ack                  78697
tcp.ack_raw              78697
tcp.checksum             78697
tcp.len                  78697
tcp.options              78697
tcp.payload              78697
tcp.seq                  78697
tcp.srcport              78697
Attack_type              78697
dtype: int64

In [70]:
temp.groupby(["Attack_type"])["Attack_type"].count()

Attack_type
Password         71035
SQL_injection     4379
Uploading         1907
XSS               1376
Name: Attack_type, dtype: int64

In [71]:
temp.columns

Index(['frame.time', 'http.request.method', 'http.request.full_uri',
       'http.request.version', 'tcp.ack', 'tcp.ack_raw', 'tcp.checksum',
       'tcp.len', 'tcp.options', 'tcp.payload', 'tcp.seq', 'tcp.srcport',
       'Attack_type'],
      dtype='object')

In [87]:
processed_data.to_csv("Concat_Processed_Data.csv", sep=",")